In [1]:
import numpy as np
from scipy.optimize import minimize_scalar
np.random.seed(666)

In [2]:
# Parameters
full_fare_price = 150
discount_price = 100

plane_capacity = 100

num_simulations = 200000  

# Distributions parameters
full_fare_mean = 56
full_fare_std = 23
discount_fare_mean = 88
discount_fare_std = 44
alpha = 0.3

def simulate_demand(mean, std, num_simulations):
    """Simulates demand with given mean and standard deviation, making sure it's non-negative"""
    demand = np.random.normal(mean, std, num_simulations)
    return np.where(demand < 0, 0, demand)  # setting negative values to 0

# Simulate demands outside the function to ensure the same set of demand realizations
full_fare_demand = simulate_demand(full_fare_mean, full_fare_std, num_simulations)
discount_demand = simulate_demand(discount_fare_mean, discount_fare_std, num_simulations)

def expected_revenue(protection_level):
    # Compute sales
    discounted_sales = np.minimum(plane_capacity - protection_level, discount_demand)
    unmet_discounted_demand = (discount_demand - discounted_sales ) * alpha
    full_fare_sales = np.minimum(plane_capacity - discounted_sales, full_fare_demand + unmet_discounted_demand)
    
    # Compute total revenue
    revenue = full_fare_price * full_fare_sales + discount_price * discounted_sales
    
    # Return the negative of the average revenue (since we want to maximize revenue, but scipy provides minimize functions)
    return -np.mean(revenue)

# Find the protection level that maximizes expected revenue
result = minimize_scalar(expected_revenue, bounds=(0, plane_capacity), method='bounded')

print(f"The optimal protection level to maximize expected revenue is: {result.x}")

The optimal protection level to maximize expected revenue is: 75.27392784088853
